In [2]:
from flask import Flask, render_template
import pymongo
# import dependencies
import time
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser

#@app.route("/scrape")
#def web_scrape():
client_connection = pymongo.MongoClient()
db = client_connection.mars_db
mars_collection = db.mars_data_entries
db.collection.remove({})
#mars_data = scrape_mars.scrape()
db.mars_collection.insert_one(mars_data)
    #return  render_template('scrape.html')

#if __name__ == "__main__":
    #app.run(debug=True)

C:\Users\Varsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  


NameError: name 'mars_data' is not defined

In [3]:


def init_browser():
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=True)

browser = init_browser()
mars_data = {}

# visit the NASA Mars News site and scrape headlines
mars_nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(mars_nasa_url)
time.sleep(1)
mars_nasa_html = browser.html
mars_nasa_soup = BeautifulSoup(mars_nasa_html, 'html.parser')

mars_news_list = mars_nasa_soup.find('ul', class_='item_list')
first_item = mars_news_list.find('li', class_='slide')
mars_nasa_headline = first_item.find('div', class_='content_title').text
mars_nasa_teaser = first_item.find('div', class_='article_teaser_body').text
mars_data["nasa_headline"] = mars_nasa_headline
mars_data["nasa_teaser"] = mars_nasa_teaser

    # visit the JPL website and scrape the featured image
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image_url)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
try:
    expand = browser.find_by_css('a.fancybox-expand')
    expand.click()
    time.sleep(1)

    featured_image_html = browser.html
    featured_image_soup = BeautifulSoup(featured_image_html, 'html.parser')

    featured_image_relative = featured_image_soup.find('img', class_='fancybox-image')['src']
    featured_image_path = f'https://www.jpl.nasa.gov{featured_image_relative}'
    mars_data["feature_image_src"] = featured_image_path
except ElementNotVisibleException:
    featured_image_path = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22076_hires.jpg'
    mars_data["feature_image_src"] = featured_image_path
        
    # visit the mars weather report twitter and scrape the latest tweet
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)
time.sleep(1)
mars_weather_html = browser.html
mars_weather_soup = BeautifulSoup(mars_weather_html, 'html.parser')

mars_weather_tweets = mars_weather_soup.find('ol', class_='stream-items')
mars_weather =  mars_weather_tweets.find('p', class_="tweet-text").text
mars_data["mars_weather_summary"] = mars_weather

# visit space facts and scrap the mars facts table
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)
time.sleep(1)
mars_facts_html = browser.html
mars_facts_soup = BeautifulSoup(mars_facts_html, 'html.parser')

mars_facts_table = mars_facts_soup.find('table', class_='tablepress tablepress-id-comp-mars blue-table')
column1 = mars_facts_table.find_all('td', class_='column-1')
column2 = mars_facts_table.find_all('td', class_='column-2')

facets = []
values = []

for row in column1:
    facet = row.text.strip()
    facets.append(facet)
        
for row in column2:
    value = row.text.strip()
    values.append(value)
        
mars_facts = pd.DataFrame({
    "Facet":facets,
    "Value":values
})

mars_facts_html = mars_facts.to_html(header=False, index=False)
mars_data["fact_table"] = mars_facts_html

    # scrape images of Mars' hemispheres from the USGS site
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
mars_hemisphere_dicts = []

for i in range(1,9,2):
    mars_hemisphere_dict = {}
        
    browser.visit(mars_hemisphere_url)
    time.sleep(1)
    mars_hemisphere_html = browser.html
    mars_hemisphere_soup = BeautifulSoup(mars_hemisphere_html, 'html.parser')
    mars_hemisphere_name_links = mars_hemisphere_soup.find_all('a', class_='product-item')
    mars_hemisphere_name = mars_hemisphere_name_links[i].text.strip('Enhanced')
        
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    mars_hemisphere_img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
        
    mars_hemisphere_img_soup = BeautifulSoup(mars_hemisphere_img_html, 'html.parser')
    mars_hemisphere_img_path = mars_hemisphere_img_soup.find('img')['src']

    mars_hemisphere_dict['title'] = mars_hemisphere_name.strip()       
    mars_hemisphere_dict['img_url'] = mars_hemisphere_img_path

    mars_hemisphere_dicts.append(mars_hemisphere_dict)

    mars_data["hemisphere_imgs"] = mars_hemisphere_dicts

    browser.quit()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=57778): Max retries exceeded with url: /session/7a1cb119638a20cdf9f254f4d75dabef/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000208422F3278>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))